In [5]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time

executable_path = {'executable_path': './chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#Scrape headlines and summaries from CNN
base_url = 'https://scholar.google.com/scholar?start='

article_list = []

n=0
page = 0
i=0


for i in range(1):  

    url= base_url + str(page) + '&q=open+cell+metal+foam&hl=en&as_sdt=0,5'
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    articles = soup.find_all("div",class_="gs_r gs_or gs_scl")
    n=0

    for article in articles:
        try:
            stats=articles[n].find('div',class_="gs_a").text.strip('&nbsp')
            dash_loc = stats.find('-')

            authors = stats[0:dash_loc-2]
            stats2=stats[dash_loc+1:]
            dash_loc_2=stats2.find('-')

            journal_date = stats[dash_loc:(dash_loc+dash_loc_2)]
            comma_loc=journal_date.find(',')
            journal=journal_date[2:comma_loc]
            date=journal_date[comma_loc+2:]
            
            title=articles[n].find('h3',class_="gs_rt").text.strip('<b>')
            pdf=articles[n].find('div',class_="gs_fl").text.strip('')
            citations=articles[n].find_all('div',class_="gs_fl")
            
            if len(citations)==2:
                citation=citations[1].text.strip("Cited by ").strip(" versions Library Search")
                citation=citation[:3].strip()
            else:
                citation=citations[0].text.strip("Cited by ").strip(" versions Library Search")
                citation=citation[:3].strip()
                pdf="No"
            summary = articles[n].find('div',class_="gs_rs").text.strip('<b>')
            summary=summary[0:(len(summary)-2)]
            
            list_item = {"Title":title,"Authors":authors,"Journal":journal,"Date":date,"Stats":stats,"Citations":citation,"Full Text Available":pdf,"Summary":summary}
            article_list.append(list_item)
            
        except (AttributeError):
            print('Missing Data')
        
        n=n+1
        
    page=page+10
    i=i+1
    print(i)
    
    time.sleep(35)

1


In [ ]:
import pandas as pd
import numpy as np

#Store data into a csv as a backup
cnn_news = pd.DataFrame(article_list)

cnn_news['Summary'].replace('\xa0...','',inplace=True)

cnn_news.to_csv("./Output Data/Articles.csv", index=False, header=True)
cnn_news.head(20)